In [1]:
fname='bazin_003'

n_tta = 6

seed = 0

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from scipy.optimize import curve_fit
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import logging
from tqdm import tqdm_notebook
import itertools
import pickle as pkl

from multiprocessing import Pool

In [3]:
import random as rn
def init_seeds(seed):

    # The below is necessary for starting Numpy generated random numbers
    # in a well-defined initial state.

    np.random.seed(seed)

    # The below is necessary for starting core Python generated random numbers
    # in a well-defined state.

    rn.seed(seed)


init_seeds(seed)

In [4]:
def curve(t, A, t0, trise, tfall):
    if tfall <= trise:
        return np.zeros(t.shape)
    res = A * np.exp(-(t - t0) / tfall) / (1 + np.exp(-(t - t0) / trise))
    return res

In [5]:
# eda_024_bazin
def get_bazin(train, object_id):
    passbands = [0, 1, 2, 3, 4, 5]
    res = pd.DataFrame()
    res['object_id'] = [object_id]
    res['bazin_trise'] = np.NaN
    for pb in passbands:
        res['bazin_tfall_%d' % pb] = np.NaN
        res['bazin_A_%d' % pb] = np.NaN
    df0 = train[train.object_id == object_id]
    if df0.hostgal_photoz.mean() == 0:
        return res
    offset = 11
    t0s = []
    trises = []
    p0s = []
    bmins = []
    bmaxs = []
    xs = []
    ys = []
    y_errs = []
    mcovs = []
    for pb in range(6):
        try:
            df = df0[(df0.object_id == object_id) & (df0.passband == pb)]
            mjd_delta_prev = (df.mjd - df.mjd.shift(1)).fillna(100).values.ravel()
            mjd_delta_next = (df.mjd.shift(-1) - df.mjd).fillna(100).values.ravel()
            x_min = df.mjd.min()
            x_max = df.mjd.max()
            y_err_mean = df.flux_err.mean()
            mjd_delta_prev = np.concatenate((100 * np.ones((offset,)),
                                mjd_delta_prev,
                                100 * np.ones((offset,)),
                              ))
            mjd_delta_next = np.concatenate((100 * np.ones((offset,)),
                                mjd_delta_next,
                                100 * np.ones((offset,)),
                              ))
            x = np.concatenate((np.linspace(x_min-500, x_min -450, offset),
                                df.mjd.values,
                                np.linspace(x_max+450, x_max+500, offset),
                              ))
            y = np.concatenate((np.random.randn(offset) * y_err_mean,
                                df.flux.values,
                                np.random.randn(offset) * y_err_mean
                               ))
            y_err = np.concatenate((y_err_mean * np.ones(offset),
                                    df.flux_err,
                                    y_err_mean * np.ones(offset)
                                  ))
            idxmax = np.argmax(y)
            t00 = x[np.argmax(y)]
            A = y.max() 
            Amin = A*1.3
            Amax = 4*A
            tstart = -5
            trise = 5
            tfall = 10
            if mjd_delta_prev[idxmax] > 50:
                tstart = -50
                Amin = 2*A
                Amax = 5*A
                trise=20
                tfall=40
            tmax = 20
            if mjd_delta_next[idxmax] > 50:
                Amin = 2*A
                Amax = 5*A
                trise=20
                tfall=40
            
            
            p0 = [(Amin + Amax) / 2, t00 + tstart, trise, tfall]
            bmin = [Amin, t00 + tstart - 100, trise/10, tfall/10]
            bmax =[Amax, t00 + tstart, trise*10, tfall*10]
            bounds = (bmin, bmax)

            def test_func(t, A, t0, trise, tfall): 
                res = curve(t, A, t0, trise, tfall)
                return res

            params, params_covariance = curve_fit(test_func, x, y, p0, y_err, 
                                                  bounds=bounds,
                                                  max_nfev=1000)
            
            median_cov = np.abs(np.median(params_covariance / A))
            
            if median_cov <= 10 and pb in range(1, 5):
                t0s.append(params[1])
            trises.append(params[2])
            p0s.append([params[0], params[3]])
            bmins.append([bmin[0], bmin[3]])
            bmaxs.append([bmax[0], bmax[3]])
            xs.append(x)
            ys.append(y)
            y_errs.append(y_err)
            mcovs.append(median_cov)
        except:
            continue
    if len(t0s) <= 1:
        return
    t00 = np.median(t0s)
    trise = np.median(trises)
    res['bazin_trise'] = trise
    for pb, p0, bmin, bmax, x, y, y_err, median_cov in zip(range(0,6), p0s, bmins, bmaxs, xs, ys, y_errs, mcovs):
        try:
            bounds = (bmin, bmax)

            def test_func(t, A, tfall): 
                res = curve(t, A, t00, trise, tfall)
                return res

            params, params_covariance = curve_fit(test_func, x, y, p0, y_err, 
                                                  bounds=bounds,
                                                  max_nfev=1000)

            
            median_cov = np.median(params_covariance / A)
            
            if median_cov <= 10:
                res['bazin_tfall_%d' % pb] = params[1]
                res['bazin_A_%d' % pb] = params[0]
        except:
            continue
    return res
    

In [6]:
def apply_bazin(df, meta):
    df = df[['object_id', 'mjd', 'passband', 'flux', 'flux_err']].merge(meta[['object_id', 'hostgal_photoz']],
                                                           how='left', on='object_id')
    agg =  [get_bazin(df, object_id) for object_id in tqdm_notebook(df.object_id.unique())]
    return pd.concat(agg, axis=0)

In [7]:
train = pd.read_csv('../input/training_set.csv')
train.head()

,object_id,mjd,passband,flux,flux_err,detected
0,615,59750.4229,2,-544.810303,3.622952,1
1,615,59750.4306,1,-816.434326,5.553370,1
2,615,59750.4383,3,-471.385529,3.801213,1
3,615,59750.4450,4,-388.984985,11.395031,1
4,615,59752.4070,2,-681.858887,4.041204,1


In [8]:
meta_cols = ['object_id', 'ddf', 'hostgal_photoz', 'target']
meta_train = pd.read_csv('../input/training_set_metadata.csv')[meta_cols]
meta_train.head()

,object_id,ddf,hostgal_photoz,target
0,615,1,0.0000,92
1,713,1,1.6267,88
2,730,1,0.2262,42
3,745,1,0.2813,90
4,1124,1,0.2415,90


In [9]:
def work_tta(param):
    (i, fname) = param
    print('starting worker', i)
    train = pd.read_csv('../input/training_set.csv')
    meta_train = pd.read_csv('../input/training_set_metadata.csv')[meta_cols]
    df = train.copy()
    if i > 0:
        init_seeds(i)
        df['flux'] += df['flux_err'] * np.random.randn(*df['flux_err'].shape)
    df = apply_bazin(df, meta_train)
    with open('../data/tta_%d_%s.pkl' % (i, fname), 'wb') as file:
        pkl.dump(df, file)  
    print('ending worker', i)
    return 'done'

In [10]:
params = [(i, fname) for i in range(11)]

if 1: 
    pool = Pool(processes=11, maxtasksperchild=1)
    ls   = pool.map( work_tta, params, chunksize=1 )
    pool.close()
else:
    ls = [work_tta(param) for param in params]

starting worker 2
starting worker 1
starting worker 0
starting worker 3
starting worker 9
starting worker 7
starting worker 5
starting worker 4
starting worker 6
starting worker 8
starting worker 10


/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in true_divide


/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in true_divide
/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: overflow encountered in multiply
/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in true_divide
/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in true_divide
/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: overflow encountered in multiply
/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: overflow encountered in multiply
/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in true_divide
/home/jfpuget/an


ending worker 0

ending worker 7

ending worker 3

ending worker 2

ending worker 1


ending worker 6
ending worker 4

ending worker 9


ending worker 8
ending worker 5

ending worker 10


In [11]:
def work_test(param):
    (i, fname) = param
    print('starting worker', i)
    with open('../input/test_chunk_%d.csv' %i, 'rb') as file:
        test = pkl.load(file)
    meta_test = pd.read_csv('../input/training_set_metadata.csv')[meta_cols]
    df = apply_bazin(test, meta_test)
    with open('../data/bazin_test_%d_%s.pkl' % (i, fname), 'wb') as file:
        pkl.dump(df, file)  
    print('ending worker', i)
    return 'done'

In [12]:
params = [(i, fname) for i in range(91)]
params.append((100, fname))

if 1: 
    pool = Pool(processes=20, maxtasksperchild=1)
    ls   = pool.map( work_test, params, chunksize=1 )
    pool.close()
else:
    ls = [work_tta(param) for param in params]

starting worker 0
starting worker 4
starting worker 7
starting worker 2
starting worker 3
starting worker 5
starting worker 1
starting worker 14
starting worker 12
starting worker 11
starting worker 13
starting worker 9
starting worker 8
starting worker 15
starting worker 10
starting worker 16
starting worker 6
starting worker 18
starting worker 17
starting worker 19


/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in true_divide
/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: overflow encountered in multiply


/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in true_divide


/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in true_divide
/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in true_divide
/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in true_divide
/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: overflow encountered in multiply
/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in true_divide
/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: overflow encountered in multiply
/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in true_divide
/home/jf

/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/scipy/optimize/_lsq/common.py:505: RuntimeWarning: invalid value encountered in greater
  mask = (g > 0) & np.isfinite(lb)
/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/scipy/optimize/_lsq/common.py:501: RuntimeWarning: invalid value encountered in less
  mask = (g < 0) & np.isfinite(ub)
/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/scipy/optimize/_lsq/common.py:505: RuntimeWarning: invalid value encountered in greater
  mask = (g > 0) & np.isfinite(lb)
/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/scipy/optimize/_lsq/common.py:501: RuntimeWarning: invalid value encountered in less
  mask = (g < 0) & np.isfinite(ub)
/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/scipy/optimize/_lsq/common.py:505: RuntimeWarning: invalid value encountered in greater
  mask = (g > 0) & np.isfinite(lb)
/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/scipy/optimize/_lsq

...



ending worker 69


/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/scipy/optimize/_lsq/common.py:501: RuntimeWarning: invalid value encountered in less
  mask = (g < 0) & np.isfinite(ub)
/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/scipy/optimize/_lsq/common.py:505: RuntimeWarning: invalid value encountered in greater
  mask = (g > 0) & np.isfinite(lb)


ending worker 67


/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/scipy/optimize/_lsq/common.py:501: RuntimeWarning: invalid value encountered in less
  mask = (g < 0) & np.isfinite(ub)
/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/scipy/optimize/_lsq/common.py:505: RuntimeWarning: invalid value encountered in greater
  mask = (g > 0) & np.isfinite(lb)
/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/scipy/optimize/_lsq/common.py:501: RuntimeWarning: invalid value encountered in less
  mask = (g < 0) & np.isfinite(ub)
/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/scipy/optimize/_lsq/common.py:505: RuntimeWarning: invalid value encountered in greater
  mask = (g > 0) & np.isfinite(lb)
/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/scipy/optimize/_lsq/common.py:501: RuntimeWarning: invalid value encountered in less
  mask = (g < 0) & np.isfinite(ub)
/home/jfpuget/anaconda3/envs/xgb8/lib/python3.6/site-packages/scipy/optimize/_lsq/co



ending worker 81
ending worker 80

ending worker 90

ending worker 87

ending worker 89

ending worker 88


ending worker 82
ending worker 83

ending worker 84

ending worker 85

ending worker 86
